In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [ ]:
import pandas as pd

# Specify the file path
algresults = "results_2000.csv"
ogfile = "indivs_Georgia20.csv"

In [ ]:
algdata = pd.read_csv(algresults, delimiter=',', header=0, names=['number','Contribid','hard2','soft2_0','soft2_1','hard3','soft3_0','soft3_1','soft3_2','hard6','soft6_0','soft6_1','soft6_2','soft6_3','soft6_4','soft6_5','hard8','soft8_0','soft8_1','soft8_2','soft8_3','soft8_4','soft8_5','soft8_6','soft8_7'])
algdata = algdata[algdata['Contribid'].str[0].str.islower()]




algdata = algdata[['Contribid','hard2','hard3','hard6', 'hard8']]

#algdata[['soft2_0', 'soft3_0', 'soft3_1']] = algdata[['soft2_0', 'soft3_0', 'soft3_1']].astype(float)

algdata[['hard2', 'hard3', 'hard6', 'hard8']] = algdata[['hard2', 'hard3', 'hard6', 'hard8']].astype(int)

print(algdata.head(10))

      Contribid  hard2  hard3  hard6  hard8
0   c0060033746      1      2      1      7
2   d0000689706      0      0      2      4
3   h1001311865      1      2      0      3
4   d0000792527      1      2      4      5
6   p0001290160      1      2      4      5
7   a0000392897      0      1      0      7
9   a0001029543      1      2      4      5
10  a0001158781      1      2      0      3
11  b9060172621      1      2      4      5
12  d1006089460      1      2      0      3


In [ ]:
# Read the CSV file into a pandas DataFrame
totalogdata = pd.read_csv(ogfile, header=0)

# Specify columns to drop
columns_to_drop = ['Cycle', 'State', 'Microfilm']

# Drop unnecessary columns
ogdata = totalogdata.drop(columns=columns_to_drop)

# Convert 'Amount' column to float
ogdata['Amount'] = ogdata['Amount'].astype(float)

# Convert selected columns to strings
ogdata[['Contribid', 'City', 'Zip', 'Recipcode', 'Gender', 'Occupation']] = ogdata[['Contribid', 'City', 'Zip', 'Recipcode', 'Gender', 'Occupation']].astype(str)
ogdata[['Recipid','Fectransid', 'Contrib', 'Orgname', 'Realcode', 'Date', 'Type', 'CmteID', 'OtherID',  'Employer', 'Source']] = ogdata[[ 'Recipid','Fectransid', 'Contrib', 'Orgname', 'Realcode', 'Date', 'Type', 'CmteID', 'OtherID',  'Employer', 'Source']].astype(str)
# Display the DataFrame
print(ogdata.head())


C:\Users\qi\AppData\Local\Temp\ipykernel_29396\2462901501.py:2: DtypeWarning: Columns (6,10,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  totalogdata = pd.read_csv(ogfile, header=0)


            Fectransid    Contribid                        Contrib    Recipid  \
0  4030220201222916370  p0003646487               GADZHIYEV, KAMAL  N00000528   
1  4030220201222916403  r0002378713                     ZHANG, WEI  N00042308   
2  4030220201222938658  j1001121876             MCELVAIN, KRISTINE  N00000528   
3  4030220201222947269  r0025197989  BREWINGTON WILLIAMS, VERONICA  C00562983   
4  4030220201222947292  q0001040790                   JONES, LLOYD  C00000935   

              Orgname Ultorg Realcode                 Date  Amount Street  \
0  [24T Contribution]    NaN    Z9500  2019-11-12 00:00:00    10.0    NaN   
1  [24T Contribution]    NaN    Z9500  2019-11-12 00:00:00   300.0    NaN   
2  [24T Contribution]    NaN    Z9500  2019-11-14 00:00:00    27.0    NaN   
3  [24T Contribution]    NaN    Z9500  2019-11-14 00:00:00    25.0    NaN   
4  [24T Contribution]    NaN    Z9500  2019-11-14 00:00:00    29.0    NaN   

          City    Zip Recipcode Type     CmteID   

In [ ]:
# Perform inner join on 'Contribid' column
merged_data = algdata.merge(ogdata, on='Contribid', how='inner')

# Drop rows where 'soft2_0' is null or NaN
merged_data = merged_data.dropna(subset=['hard2'])

# Aggregate the merged DataFrame
merged_data = merged_data.groupby('Contribid').agg({
    'Amount': 'sum',
    'hard2': 'first',
    'hard3': 'first',
    'hard6': 'first',
    'hard8': 'first',
    'City': 'first',
    'Zip': 'first',
    'Recipcode': 'first',
    'Gender': 'first',
    'Occupation': 'first',
    'Recipid': 'first',
    'Fectransid': 'first',
    'Contrib': 'first',
    'Orgname': 'first',
    'Ultorg': 'first',
    'Realcode': 'first',
    'Date': 'first',
    'Street': 'first',
    'Type': 'first',
    'CmteID':'first',
    'OtherID': 'first',
    'Employer': 'first',
    'Source': 'first'
}).reset_index()

# Display the aggregated DataFrame
print(merged_data)

df = merged_data


         Contribid     Amount  hard2  hard3  hard6  hard8         City    Zip  \
0      a0000051892    35000.0      0      1      1      7     LAGRANGE  30241   
1      a0000057574    11200.0      0      0      1      7  THOMASVILLE  31799   
2      a0000058568     3500.0      1      2      5      1         ROME  30165   
3      a0000087810    12350.0      1      2      1      3      ATLANTA  30308   
4      a0000087824  1669452.0      0      0      4      5      ATLANTA  30327   
...            ...        ...    ...    ...    ...    ...          ...    ...   
10987  r0025253382     4300.0      0      1      4      5      ATLANTA  30345   
10988  r0025255955     5000.0      0      0      2      4      ATLANTA  30305   
10989  r0025258632   302228.0      1      2      4      6      ACWORTH  30101   
10990  r0025272782     7302.0      0      0      4      5      ATLANTA  30318   
10991  r0025275280    10500.0      1      1      5      6      ATLANTA  30342   

      Recipcode Gender  ...

In [ ]:
#merged_data.to_csv("mergeddata.csv", index=False)

In [ ]:
#set the labels for the decision tree and build a quantitative dataset for the decision tree to use in splits
###### Change this as needed
y = df['hard8']
######
X = pd.DataFrame()
X['Amount'] = df['Amount']
X['Urban'] = [1 if x in ['ATLANTA', 'AUGUSTA', 'COLUMBUS', 'MACON', 'SAVANNAH', 'ATHENS'] else 0 for x in df['City']]
X['Zip'] = df['Zip']
X['Democrat'] = [1 if x[0] == 'D' else 0 for x in df['Recipcode']]
X['Republican'] = [1 if x[0] == 'R' else 0 for x in df['Recipcode']]
X['Sex'] = [1 if x == 'M' else 0 for x in df['Gender']]
X['Time'] = [(int(x[0:4])%2019)*365+int(x[5:7])*31+int(x[8:10]) for x in df['Date']]

In [ ]:
#Create and train decision tree
X_train, X_test, y_train, y_test = train_test_split(X, y)
clf = DecisionTreeClassifier(criterion='entropy',max_depth=3)
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3)

In [ ]:
#Find the optimal parameters
param_grid = {'min_samples_leaf': [40, 45, 50, 55], 'max_depth': [5, 6, 7, 8, 9], 'min_samples_split': [0.05, 0.1, 0.2, 0.3]}
gscv_clf = GridSearchCV(estimator=clf, param_grid=param_grid)
gscv_clf.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(criterion='entropy', max_depth=3),
             param_grid={'max_depth': [5, 6, 7, 8, 9],
                         'min_samples_leaf': [40, 45, 50, 55],
                         'min_samples_split': [0.05, 0.1, 0.2, 0.3]})

In [ ]:
#Display them and the achieved accuracy
print(gscv_clf.best_params_)
print(gscv_clf.best_score_)

{'max_depth': 6, 'min_samples_leaf': 40, 'min_samples_split': 0.1}
0.26965047954924143


In [ ]:
#Understand the structure of the tree to see the characteristics of each cluster
#Code comes directly form scikit documentation for decision tree
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
values = clf.tree_.value

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, 0)]
while len(stack) > 0:
    node_id, depth = stack.pop()
    node_depth[node_id] = depth

    is_split_node = children_left[node_id] != children_right[node_id]
    if is_split_node:
        stack.append((children_left[node_id], depth + 1))
        stack.append((children_right[node_id], depth + 1))
    else:
        is_leaves[node_id] = True

print(
    "The binary tree structure has {n} nodes and has "
    "the following tree structure:\n".format(n=n_nodes)
)
for i in range(n_nodes):
    if is_leaves[i]:
        print(
            "{space}node={node} is a leaf node with value={value}.".format(
                space=node_depth[i] * "\t", node=i, value=values[i]
            )
        )
    else:
        print(
            "{space}node={node} is a split node with value={value}: "
            "go to node {left} if X[:, {feature}] <= {threshold} "
            "else to node {right}.".format(
                space=node_depth[i] * "\t",
                node=i,
                left=children_left[i],
                feature=feature[i],
                threshold=threshold[i],
                right=children_right[i],
                value=values[i],
            )
        )

The binary tree structure has 15 nodes and has the following tree structure:

node=0 is a split node with value=[[0.47828724 0.52171276]]: go to node 1 if X[:, 3] <= 0.5 else to node 8.
	node=1 is a split node with value=[[0.4336976 0.5663024]]: go to node 2 if X[:, 0] <= 5599.5 else to node 5.
		node=2 is a split node with value=[[0.29382716 0.70617284]]: go to node 3 if X[:, 4] <= 0.5 else to node 4.
			node=3 is a leaf node with value=[[0.21369863 0.78630137]].
			node=4 is a leaf node with value=[[0.53553719 0.46446281]].
		node=5 is a split node with value=[[0.52553364 0.47446636]]: go to node 6 if X[:, 6] <= 690.5 else to node 7.
			node=6 is a leaf node with value=[[0.4765413 0.5234587]].
			node=7 is a leaf node with value=[[0.63814617 0.36185383]].
	node=8 is a split node with value=[[0.60766682 0.39233318]]: go to node 9 if X[:, 0] <= 12337.5 else to node 12.
		node=9 is a split node with value=[[0.6425 0.3575]]: go to node 10 if X[:, 0] <= 3915.0 else to node 11.
			node=10 